In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
from torch.utils.data import DataLoader, Dataset
import torch
from data_import import *

In [2]:
oregon_data_dict = oregon_import()

file import: 100%|██████████| 3/3 [00:00<00:00,  4.06it/s]


In [3]:
# wa_df = oregon_data_dict['train_timeseries'].iloc[4:,:].reset_index(inplace=False, drop=True)
wa_df = oregon_data_dict['train_timeseries'].copy()
wa_df = wa_df[wa_df['fips']==41067]
wa_df.drop(columns=['fips'],inplace=True)
wa_df = wa_df.iloc[4:,:]
wa_df = wa_df.iloc[:-4,:]
wa_df['date'] = wa_df['date'].map(pd.Timestamp.timestamp)
wa_df.reset_index(inplace=True,drop=True)

In [20]:
X_1 = wa_df.iloc[:7, (wa_df.columns != 'score') & (wa_df.columns != 'date')]
y_1 = wa_df.iloc[6, wa_df.columns == 'score']
date_1 = wa_df.iloc[:7, wa_df.columns == 'date']

In [6]:
X_2 = wa_df.iloc[7:14, (wa_df.columns != 'score') & (wa_df.columns != 'date')]
y_2 = wa_df.iloc[13, wa_df.columns == 'score']
date_2 = wa_df.iloc[7:14, wa_df.columns == 'date']

In [4]:
class DroughtDataset(Dataset):
    """
    Readying Drought dataset for model.
    """
    def __init__(self, df):
        self.X = torch.tensor(df.iloc[:, (df.columns != 'score') & (df.columns != 'date')].values)
        self.y = torch.tensor(df.iloc[:, df.columns == 'score'].dropna().values)
        self.date = np.array(df.iloc[:, df.columns == 'date'].values)

    def __len__(self):
        return int(len(self.y)/7)
    
    def __getitem__(self, index):
        output_X = self.X[7*index:7+7*index]
        output_y = self.y[6 + 7*index]
        output_date = self.date[7*index:7+7*index]

        return output_X, output_y, output_date

In [5]:
wa_dataset = DroughtDataset(wa_df)

In [32]:
x, y, date = wa_dataset[0]

In [6]:
wa_data_loader = DataLoader(wa_dataset, batch_size = 16, shuffle=True)

In [11]:
for batch, (soil_info, drought_rating, date) in enumerate(wa_data_loader):
    if batch == 0:
        break

In [9]:
import torch.nn as nn
from torch import optim
torch.manual_seed(1)

In [7]:
# hyperparameters
sequence_len = 7
input_len = 18 # number of explanatory variable columns
hidden_size = 128
num_layers = 2
num_epochs = 10
learning_rate = 1e-2

In [10]:
class LSTM_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim = 1):
        super(LSTM_Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h_0=None, c_0=None):
        # managing hidden states and cell states
        if h_0 is None or c_0 is None:
            h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
            c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        
        out, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        out = self.output_layer(out[:, -1, :])
        return out, h_n, c_n

In [11]:
model = LSTM_Model(input_dim = input_len, hidden_dim = hidden_size, num_layers = num_layers)
print(model)

LSTM_Model(
  (lstm): LSTM(18, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=1, bias=True)
)


In [13]:
loss_function = nn.MSELoss();
optimizer = optim.Adam(model.parameters(), lr=learning_rate);

In [14]:
def train(num_epochs: int, model: nn.Module, loss_fn, optimizer, train_data_loader):
    """
    trains the model.

    Parameters
    ----------
    'num_epochs' : int
    'model' : nn.Module
    'train_data_loader'
    """

    total_steps = len(train_data_loader)

    for epoch in range(num_epochs):
        for batch, (soil_info, drought_rating, date) in enumerate(train_data_loader):
            output = model(soil_info)
            loss = loss_fn(output, drought_rating)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch+1)%100 == 0:
                print(f'Epoch: {epoch+1}; Batch: {batch+1} / {total_steps}; Loss: {loss.item():>4f}') # rounding the loss
    return None

In [15]:
train(num_epochs=num_epochs, model=model, loss_fn=loss_function, optimizer=optimizer, train_data_loader=wa_data_loader)

AttributeError: 'tuple' object has no attribute 'size'